In [1]:
import numpy as np
import torch
from torch import nn
import time
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


<h3>DAtaset creation</h3>

In [3]:
data = np.load("data/images.npy")
labels = torch.FloatTensor(np.load("data/labels.npy")).to(device)

data_b = torch.FloatTensor(np.expand_dims(data,axis=1)).to(device)
print(f"data: {data_b.shape}, labels: {labels.shape}")

data: torch.Size([18000, 1, 150, 150]), labels: torch.Size([18000, 2])


In [4]:
class ClockDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [5]:
# Create main dataset
clock_dataset = ClockDataset(data_b, labels)
train_data, test_data, val_data = torch.utils.data.random_split(clock_dataset, [14000,3000,1000])

# Split dataset into train, test and validation sets
train_data = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_data = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
val_data = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)

In [6]:
class NN_regression(nn.Module):
    """ Convolution model that returns one value as output.
    """
    def __init__(self, input_channels, h, w, n_outputs):
        super(NN_regression, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=5),
            nn.MaxPool2d(),
        )
        self.hidden_layers = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=2),
            nn.ReLU(),
            nn.Flatten()
        )


        self.output_layer = nn.Sequential(
            nn.Linear(304704, 256),
            nn.ReLU(),
            nn.Linear(256, n_outputs)
            
        )


    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x


In [7]:
model = NN_regression(input_channels=1,h=150,w=150, n_outputs=2).to(device)

In [8]:
#print(example.shape)
tensor = data_b[:10]
print(tensor.shape)
ret = model.forward(tensor)
print(ret.shape)

torch.Size([10, 1, 150, 150])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x609408 and 304704x256)

<h3>Creating loss function</h3>

In [ ]:
def common_sense_loss(prediction, target):
    """ Defines the common sense error in minutes.
    """

    # hour common sense error in minutes
    hour_start_idx = torch.minimum(prediction[0], target[0])
    hour_end_idx = torch.maximum(prediction[0], target[0])
    hour_dist_mins = (hour_end_idx - hour_start_idx)*60

    # minutes common sense error
    min_start_idx = torch.minimum(prediction[1], target[1])
    min_end_idx = torch.maximum(prediction[1], target[1])
    mins_dist_mins = (min_end_idx - min_start_idx)

    # return total common sense error in minutes
    return torch.mean(hour_dist_mins + mins_dist_mins)

a = torch.Tensor([[1,4,2,3],[14,28,36,1]])
b = torch.Tensor([[1,3,2,4],[15,29,36,1]])


In [ ]:
class CommonSenseError(nn.Module):
    def __init__(self):
        super(CommonSenseError, self).__init__()

    def forward(self, prediction, target):
        # hour common sense error in minutes
        hour_start_idx = torch.minimum(prediction[0], target[0])
        hour_end_idx = torch.maximum(prediction[0], target[0])
        hour_dist_mins = (hour_end_idx - hour_start_idx)*60

        # minutes common sense error
        min_start_idx = torch.minimum(prediction[1], target[1])
        min_end_idx = torch.maximum(prediction[1], target[1])
        mins_dist_mins = (min_end_idx - min_start_idx)

        # return total common sense error in minutes
        return torch.mean(hour_dist_mins + mins_dist_mins)

In [ ]:
loss = CommonSenseError()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss, optimizer):
    model.train()
    for (X, y) in enumerate(dataloader):
        #X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = common_sense_loss(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train loss: {loss:>7f}")

In [ ]:
def test(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            #X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss
    test_loss /= num_batches
    print(f"Test avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, loss, optimizer)
    test(test_data, model, loss)
end_time = time.time()

Epoch 1
-------------------------------
Train loss: 400.500000
Test avg loss: 1078.537231 

Epoch 2
-------------------------------
Train loss: 1074.000000
Test avg loss: 954.930847 

Epoch 3
-------------------------------
Train loss: 1118.000000
Test avg loss: 996.170166 

Epoch 4
-------------------------------
Train loss: 308.500000
Test avg loss: 1078.984009 

Epoch 5
-------------------------------
Train loss: 775.500000
Test avg loss: 1125.797852 



In [ ]:
print(f"Done in {end_time - start_time}")

Done in 84.54817366600037


In [ ]:
one = 122.95
two = 124.95
tree = 84.58
four = 84.43